First and formost, create a series as example:

首先，创建一个多层目录series用于演示

In [29]:
import pandas as pd
import numpy as np

arrays = [['bar', 'bar', 'baz', 'baz', 'foo', 'foo', 'qux', 'qux'], ['one', 'two', 'one', 'two', 'one', 'two', 'one', 'two']]
tuples = list(zip(*arrays))
index = pd.MultiIndex.from_tuples(tuples, names=['first', 'second'])
s = pd.Series(np.random.randn(8), index=index)
print(s)

first  second
bar    one       1.216353
       two      -0.364467
baz    one      -1.488360
       two      -0.987053
foo    one      -1.422452
       two      -1.446716
qux    one       0.928029
       two      -0.078733
dtype: float64


If we simply sort by python built-in functions, it will looks like that:

如果我们只是单纯地用python内置库进行排序，会变成这样：

In [30]:
s.sort_index(level = 'first').sort_values()

first  second
baz    one      -1.488360
foo    two      -1.446716
       one      -1.422452
baz    two      -0.987053
bar    two      -0.364467
qux    two      -0.078733
       one       0.928029
bar    one       1.216353
dtype: float64

It is not we want. What we need is to lock the index 'first' first, do sorting inside the index and than sort the whole bunch of data with different 'first'.

So convert series into dataframe first by using reset_index(). Dataframe provide a function also called sort_values(), but because of the data type 'Dataframe' does not have multi-indexes, the different indexes in series will be considered as values in dataframe.

这并不是我们想要的结果（first那列乱了！！），我们需要先锁定first那列，然后对后面那堆数值进行排序，然后再将拥有相同first的项作为一个整体，对first进行排序（真正的排序结果在本文最后）。

所以要把series先转换成dataframe(用reset_index()这个函数转的)，这样就可以用reset_index方程根据排序优先级进行排序。因为dataframe没有多层目录这个东西，所有的目录名都被当成变量处理。我们把目录的优先级设置成最高就不会将目录打乱了

In [31]:
s_df = s.reset_index()
print(s_df)
print('\n Now the datatype of \'s_df\' is:')
print(type(s_df))

first second         0
0   bar    one  1.216353
1   bar    two -0.364467
2   baz    one -1.488360
3   baz    two -0.987053
4   foo    one -1.422452
5   foo    two -1.446716
6   qux    one  0.928029
7   qux    two -0.078733

 Now the datatype of 's_df' is:
<class 'pandas.core.frame.DataFrame'>


In [32]:
s_sorted_df = s_df.sort_values(by = ['first',0])
print(s_sorted_df)

first second         0
1   bar    two -0.364467
0   bar    one  1.216353
2   baz    one -1.488360
3   baz    two -0.987053
5   foo    two -1.446716
4   foo    one -1.422452
7   qux    two -0.078733
6   qux    one  0.928029


Now use this dataframe to rebuild a series, the new series is the outcome we want

现在就用这个排好序的Dataframe去重新建立Series，下面那堆实在是很麻烦，没有能直接转换的内置函数。

In [33]:
index1 = s_sorted_df.loc[:,'first'].tolist()
index2 = s_sorted_df.loc[:,'second'].tolist()
sorted_values = s_df.loc[:,0]

print('Data Type of the data under index1:')
print(type(index1))
print('\n')
print('Data Type of the data under index2:')
print(type(index2))
print('\n')
print('Data Type of the data under sorted_values:')
print(type(sorted_values))

Data Type of the data under index1:
<class 'list'>


Data Type of the data under index2:
<class 'list'>


Data Type of the data under sorted_values:
<class 'pandas.core.series.Series'>


values should be converted to numpy-array otherwise it cannot be used to from a series

需要注意的是，数值那列需要转换成numpy.array格式才能作为series的数值

In [34]:
sorted_values = np.array(sorted_values)
print('Now the datatype of values is:')
print(type(sorted_values))

Now the datatype of values is:
<class 'numpy.ndarray'>


Now assemble everything to form a series

然后把几个目录和数值弄到一起就好了。

In [35]:
sorted_index = [index1,index2]
sorted_s = pd.Series(sorted_values,index=sorted_index)
print(sorted_s)

bar  two    1.216353
     one   -0.364467
baz  one   -1.488360
     two   -0.987053
foo  two   -1.422452
     one   -1.446716
qux  two    0.928029
     one   -0.078733
dtype: float64


That is what we want.

完工